In [ ]:
import numpy as np
import pandas as pd
from skimage.io import imread, imread_collection

from sklearn.datasets import fetch_olivetti_faces
from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.linear_model import ElasticNet, BayesianRidge

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, Dense, Dropout
from tensorflow.keras.activations import sigmoid
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.metrics import binary_accuracy

from matplotlib import pyplot as plt
from skimage.transform import resize
from skimage.color import rgb2gray
from skimage import img_as_float

RGB Faces Dataset

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d gasgallo/faces-data-new

In [ ]:
! unzip faces-data-new.zip

Grayscale Faces Dataset

In [ ]:
dataset = fetch_olivetti_faces(return_X_y=True)
faces = dataset[0]

# 1. Principle Component Analysis based Image Compression for Grayscale Faces

In [ ]:
partition = (int)(len(faces)*0.6)
train, test = faces,faces[partition:]
img_test = np.reshape(test,(-1,64,64))

In [ ]:
for i in [200]:
  pca = PCA(n_components=i)
  pca.fit(train)
  ytest = pca.transform(test)
  ytest = pca.inverse_transform(ytest)
  print(f'MSE for {i} eigenfaces : {np.sqrt(np.mean(np.square(test-ytest))):.5f}')
ytest = np.reshape(ytest,newshape=(-1,64,64))

In [ ]:
idx = np.random.randint(len(ytest))
plt.subplot(141)
plt.imshow(ytest[idx],cmap='gray')
plt.subplot(142)
plt.imshow(img_test[idx],cmap='gray')
plt.subplot(143)
idx = np.random.randint(len(ytest))
plt.imshow(ytest[idx],cmap='gray')
plt.subplot(144)
plt.imshow(img_test[idx],cmap='gray')

plt.show()
plt.clf()

# 2. Classification on PCA data

In [ ]:
faces = imread_collection(['/content/Faces/*.jpg'],conserve_memory=False)
np.random.shuffle(faces.files)
X = list()
for i,file in enumerate(faces.files):
  X.append(resize(rgb2gray(img_as_float(faces.load_func(file))),(64,64)).flatten())

Y = np.zeros(len(faces.data))
for i,file in enumerate(faces.files):
  string = file.split('_');
  Y[i] = int(string[1])

In [ ]:
X = np.array(X)
partition = (int)(len(faces.data)*0.8)
xtrain, xtest = np.array(X[:partition]),np.array(X[partition:])
ytrain, ytest = Y[:partition],Y[partition:]

In [ ]:
n_components=200
pca = PCA(n_components)
pca.fit(xtrain)
test = pca.transform(xtest)
test = pca.inverse_transform(test)
print(f'MSE for {i} eigenfaces : {np.sqrt(np.mean(np.square(test-xtest))):.5f}')
test = np.reshape(test,newshape=(-1,64,64))

In [ ]:
idx = np.random.randint(len(test))
plt.subplot(141)
plt.imshow(test[idx],cmap='gray')
plt.subplot(142)
plt.imshow(np.reshape(xtest[idx],(64,64)),cmap='gray')
plt.subplot(143)
idx = np.random.randint(len(ytest))
plt.imshow(test[idx],cmap='gray')
plt.subplot(144)
plt.imshow(np.reshape(xtest[idx],(64,64)),cmap='gray')

plt.show()
plt.clf()

In [ ]:
xtrain = pca.transform(xtrain)
xtrain.shape

In [ ]:
ytrain = np.reshape(ytrain,(-1,1))
ytrain.shape

In [ ]:
xtest = pca.transform(xtest)
xtest.shape

In [ ]:
ytest = np.reshape(ytest,(-1,1))
ytest.shape

## 2.1 Elastic Net Classifier

In [ ]:
genderModel = ElasticNet()
genderModel.fit(X=xtrain,y=ytrain)

ElasticNet()

In [ ]:
res = genderModel.predict(xtest)
min = np.min(res)
max = np.max(res)
res = (res-min)/(max-min)

In [ ]:
1-np.mean(np.abs(np.round(res)-ytest.ravel()))

## 2.2 Naive Bayes Classifier

In [ ]:
genderModel = BayesianRidge()
genderModel.fit(X=xtrain,y=ytrain.ravel())

In [ ]:
res = genderModel.predict(xtest)
min = np.min(res)
max = np.max(res)
res = (res-min)/(max-min)

In [ ]:
1-np.mean(np.abs(np.round(res)-ytest.ravel()))

## 2.3 Deep Learning Classifier to identify Gender

In [ ]:
genderModel = Sequential([
  Input(shape=(n_components,)),
  Dense(units=n_components//4,activation=sigmoid),
  Dense(units=n_components//16,activation=sigmoid),
  Dense(units=1,activation=sigmoid),
])

genderModel.compile(optimizer=Adam(),loss=binary_crossentropy,metrics=[binary_accuracy])

In [ ]:
genderModel.summary()

In [ ]:
genderModel.fit(x=xtrain,y=ytrain,batch_size=32,epochs=50,validation_data=(xtest,ytest))

# 3. Principle Component Analysis based Image Compression for RGB Faces

In [ ]:
ic = imread_collection(['/content/images/images/*.jpg'],conserve_memory=False)

In [ ]:
rgb_train = list()

for i,file in enumerate(ic.files):
  ic.data[i] = img_as_float(ic.load_func(file))
  ic.data[i] = resize(ic.data[i],(100,100))
  rgb_train.append(ic.data[i].flatten())


In [ ]:
rgb_train = np.array(rgb_train)

In [ ]:
rgb_pca = IncrementalPCA(400)
rgb_pca.fit(rgb_train)

In [ ]:
i=np.random.randint(len(rgb_train))
j=np.random.randint(len(rgb_train))

img = ic.data[i]
transformed_img = rgb_pca.transform(np.reshape(img.flatten(),(1,-1)))
t_img1 = rgb_pca.inverse_transform(transformed_img)
t_img1 = np.reshape(t_img1,(100,100,3))

img = ic.data[j]
transformed_img = rgb_pca.transform(np.reshape(img.flatten(),(1,-1)))
t_img2 = rgb_pca.inverse_transform(transformed_img)
t_img2 = np.reshape(t_img2,(100,100,3))

plt.subplot(141)
plt.imshow(ic.data[i],cmap='brg')
plt.subplot(142)
plt.imshow(t_img1,cmap='brg')
plt.subplot(143)
plt.imshow(ic.data[j],cmap='brg')
plt.subplot(144)
plt.imshow(t_img2,cmap='brg')
plt.imshow(t_img2,cmap='brg')
plt.show()
plt.clf()